In [ ]:
import biogeme
from biogeme.database import Database
import pandas as pd

swissmetro = pd.read_csv("examples/swissmetro/swissmetro.dat", sep="\t")
database = Database("swissmetro", swissmetro)
globals().update(database.variables)

# Removing some observations
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
database.remove(exclude)
database.data["CHOICE"] -= 1

In [4]:
import numpy as np
import aesara
import aesara.tensor as aet
from biogeme.database import Database
import pandas as pd
from biogeme_aesara import DatabaseShared, BetaShared


swissmetro = pd.read_csv("examples/swissmetro/swissmetro.dat", sep="\t")
db = DatabaseShared("swissmetro", swissmetro, choiceVar="CHOICE")
globals().update(db.variables)


# Removing some observations
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
db.remove(exclude)
db.data["CHOICE"] -= 1 # set the first choice to 0


b_cost = BetaShared("b_cost", 0., None, None, 0)
b_time = BetaShared("b_time", 0., None, None, 0)
asc_train = BetaShared("asc_train", 0., None, None, 0)
asc_car = BetaShared("asc_car", 0., None, None, 0)
asc_sm = BetaShared("asc_sm", 0., None, None, 0)

params = [b_cost(), b_time(), asc_train(), asc_car(), asc_sm()]

print(asc_sm.status)

# load all BetaShared Python variables into params
print(b_cost() + 1) # b_cost(0.0) Aesara style
print(b_cost + 1) # (b_cost(0.0) + `1`) Biogeme style
print(b_cost * TRAIN_CO.x)
print(type(asc_train))

0
Elemwise{add,no_inplace}.0
(b_cost(0.0) + `1`)
Elemwise{mul,no_inplace}.0
<class 'biogeme_aesara.BetaShared'>


In [5]:
# define the utility functions
U_1 = b_cost * TRAIN_CO.x + b_time * TRAIN_TT.x + asc_train
U_2 = b_cost * CAR_CO.x + b_time * CAR_TT.x + asc_car
U_3 = b_cost * SM_CO.x + b_time * SM_TT.x + asc_sm

print("U_1", type(U_1)) # returns aesara.tensor.var.TensorVariable
print(db.data.TRAIN_CO.shape) # returns a vector of variables
print(b_cost.sharedVar.eval().shape)

U_1 <class 'aesara.tensor.var.TensorVariable'>
(6768,)
()


In [6]:
utility_vector = aet.concatenate([U_1, U_2, U_3], axis=1)
prob = aet.nnet.softmax(utility_vector, axis=None)
y_pred = aet.argmax(prob, axis=1)
neg_loglike = -aet.mean(aet.log(prob)[aet.arange(CHOICE.y.shape[0]), CHOICE.y])
error = aet.mean(aet.neq(y_pred, CHOICE.y))
grads = aet.grad(neg_loglike, params)

# print("b_cost_shape", b_cost.sharedVar.shape.eval())

# hess, updates = aesara.scan(
# 	lambda i, y, x: aet.grad(
# 		y[i],
# 		x,
# 		consider_constant=None,
# 		disconnected_inputs="ignore",
# 	),
# 	sequences=range(len(grads)),
# 	non_sequences=[grads, params],
# )

In [25]:
hess = aet.as_tensor_variable(np.zeros((len(grads), len(grads))))
for i in range(len(grads)):
	hess = aet.set_subtensor(x=hess[i,:], 
        y=aet.grad(
            cost=grads[i], wrt=params, consider_constant=None,
            disconnected_inputs="ignore"))

bhhh = aet.outer(aet.as_tensor_variable(grads), aet.as_tensor_variable(grads).T)
double1 = aet.grad(grads[0], params)

# double2 = aet.grad(g_b_time, [b_time(), b_cost()])

lr = 0.001
updates = [(param, param - lr * grad) for param, grad in zip(params, grads)]

loglikelihood_estimation = aesara.function(
    inputs=db.inputs(),
    outputs=neg_loglike,
    updates=updates,
    on_unused_input="ignore",
)

output_bhhh = aesara.function(
    inputs=db.inputs(),
    outputs=bhhh,
    on_unused_input="ignore",
)

output_errors = aesara.function(
    inputs=db.inputs(),
    outputs=error,
    on_unused_input="ignore",
)

output_estimated_betas = aesara.function(
    inputs=db.inputs(),
    outputs=params,
    on_unused_input="ignore",
) 

for i in range(500):
    output = loglikelihood_estimation(*(db.input_data()))
    if i % 100 == 0:
        print("iteration {}: {}".format(i, output))
    if i % 200 == 0:
        print(output_errors(*(db.input_data())))
# test_bh = bh(*(db.get_x_data() + db.get_y_data()))
# print(test_bh)
test_betas = output_estimated_betas(*(db.input_data()))
estimated_betas = {}
for n, p in enumerate(params):
    estimated_betas[p] = test_betas[n].tolist().round(3)
print(estimated_betas)

iteration 0: 1123.20703125
0.6282505910165485
iteration 100: 456.2341613769531
iteration 200: 1585.26953125
0.630466903073286
iteration 300: 927.25341796875
iteration 400: 280.29791259765625
0.6529255319148937


AttributeError: 'float' object has no attribute 'round'

In [29]:
for n, p in enumerate(params):
    estimated_betas[p] = np.round(test_betas[n].tolist(), 3)
print(estimated_betas)

{b_cost: -4.859, b_time: -1.258, asc_train: 0.268, asc_car: -0.702, asc_sm: 0.434}


In [24]:
from scipy import linalg
output = f(*(db.get_x_data() + db.get_y_data()))
np.set_printoptions(suppress=True)
print("hessian matrix:")
print(output)
varcovar = -linalg.pinv(np.nan_to_num(output))
stdErr = []
for i in range(len(params)):
    if varcovar[i, i] < 0:
        stdErr.append(np.finfo(float).max)
    else:
        stdErr.append(np.sqrt(varcovar[i, i]))
print("standard errors:")
print(stdErr)

d = np.diag(varcovar)
if (d > 0).all():
    diag = np.diag(np.sqrt(d))
    diagInv = linalg.inv(diag)
    correlation = diagInv.dot(varcovar.dot(diagInv))
else:
    correlation = np.full_like(varcovar, np.finfo(float).max)

print("correlation matrix:")
print(correlation)

test_bh = bh(*(db.get_x_data() + db.get_y_data()))
# test_bh = linalg.inv(np.nan_to_num(test_bh))
# print("BHHH:")
# print(test_bh)
robust_varCovar = varcovar.dot(test_bh.dot(varcovar))
robStdErr = []
for i in range(len(params)):
    if robust_varCovar[i, i] < 0:
        robStdErr.append(np.finfo(float).max)
    else:
        robStdErr.append(np.sqrt(robust_varCovar[i, i]))
rd = np.diag(robust_varCovar)
if (rd > 0).all():
    diag = np.diag(np.sqrt(rd))
    diagInv = linalg.inv(diag)
    robust_correlation = diagInv.dot(robust_varCovar.dot(diagInv))
else:
    robust_correlation = np.full_like(robust_varCovar, np.finfo(float).max)

print("rob correlation matrix:")
print(robust_correlation)   

test_betas = estimated_betas(*(db.get_x_data() + db.get_y_data()))
print(np.asarray(test_betas))

NameError: name 'f' is not defined